In [1]:
import os
import json
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, confusion_matrix
from scipy.spatial import distance
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
import cv2
from ultralytics import YOLO
import mediapipe as mp

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### yolo, GRU 동시 실행

In [14]:
# 랜드마크 인덱스 정의 (예: 코, 왼쪽 어깨, 오른쪽 어깨 등)
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]  # 총 11개 랜드마크

# YOLO 모델 로드
yolo_model = YOLO('D:\\project\\prjvenv\\runs\\detect\\human_fall_s30\\weights\\best.pt')

# GRU 모델 정의 및 로드 
class GRUModel(torch.nn.Module):
    def __init__(self, input_size=27):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size = 64
        self.num_layers = num_layers = 2
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True,
                          dropout=0.5)
        self.fc = nn.Linear(hidden_size, 3)  # output_size를 직접 지정합니다.
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

# GRU 모델 로드
input_size = len(LANDMARKS) * 2 + 5   # 랜드마크 (22) + 바운딩박스 정보 (4) + 속도 (1) + 클래스 (1) 
gru_model = GRUModel(input_size=input_size)  
gru_model.load_state_dict(torch.load('D:\\project\\prjvenv\\GRU\\best_GRU_model_2.pt', map_location=torch.device('cpu')))
gru_model.eval()

# 클래스 이름 정의
class_names = {0: 'Normal', 1: 'Fall', 2: 'Danger'}

def calculate_head_upper_body_speed(keypoints, prev_keypoints):
    h = np.array([keypoints[0, 0], keypoints[0, 1]])   # 머리 좌표
    l = np.array([keypoints[11, 0], keypoints[11, 1]])  # 왼쪽 어깨 좌표
    r = np.array([keypoints[12, 0], keypoints[12, 1]])  # 오른쪽 어깨 좌표

    # 이전 프레임의 좌표
    prev_h = np.array([prev_keypoints[0, 0], prev_keypoints[0, 1]])
    prev_l = np.array([prev_keypoints[11, 0], prev_keypoints[11, 1]])
    prev_r = np.array([prev_keypoints[12, 0], prev_keypoints[12, 1]])

    # 현재 프레임과 이전 프레임의 상체 중심 계산
    center_new = (h + l + r) / 3
    center_prev = (prev_h + prev_l + prev_r) / 3

    # 유클리드 거리 계산 (속도)
    speed = distance.euclidean(center_new, center_prev)
    return speed

def process_landmarks(landmarks): 
    selected_landmarks = landmarks[LANDMARKS]   # 지정된 랜드마크 선택 
    return selected_landmarks[:, :2].flatten()   # (x,y) 좌표 반환

def detect_fall(landmarks, prev_landmarks):
    speed = calculate_head_upper_body_speed(landmarks, prev_landmarks) if prev_landmarks is not None else 0
    processed_landmarks = process_landmarks(landmarks)

    # 비디오 프레임을 YOLO 입력 크기로 리사이즈
    resized_frame = cv2.resize(frame, (640, 640))
    
    # YOLO를 사용하여 바운딩 박스 예측
    results = yolo_model(resized_frame)
   
    # YOLO 예측 결과에서 바운딩 박스 정보 가져오기 
    bbox_info=results[0].boxes.xyxy.cpu().numpy() if results and len(results[0].boxes) > 0 else None
    
    if bbox_info is None or len(bbox_info) == 0:
       print("No bounding boxes detected.")
       return None , None

    # 첫 번째 바운딩 박스 정보 가져오기 (여러 개가 있을 경우 첫 번째만 사용)
    x1 , y1 , x2 , y2=bbox_info[0]  
    
    # 바운딩 박스 좌표를 원본 프레임에 맞게 변환 (640x640에서 원본 크기로)
    original_width = frame.shape[1]
    original_height = frame.shape[0]
    x1, y1, x2, y2 = bbox_info[0]
    
    x1 *= original_width / 640.0
    x2 *= original_width / 640.0
    y1 *= original_height / 640.0
    y2 *= original_height / 640.0
    print(f"Transformed coordinates: {(x1, y1, x2, y2)}")

    bbox_width=x2 - x1 
    bbox_height=y2 - y1  
   
    bbox_ratio=bbox_width / bbox_height if bbox_height !=0 else float('inf')
   
    # 클래스 결정 
    if bbox_ratio <=1.3:
       bbox_class=0   # Normal 
    elif bbox_ratio <=1.7:
       bbox_class=2   # Danger 
    else:
       bbox_class=1   # Fall 

    # 입력 데이터 구성 
    input_data=np.concatenate([processed_landmarks,
                               [bbox_width,
                                bbox_height,
                                bbox_ratio,
                                speed,
                                bbox_class]])

    print(f"Input data length: {len(input_data)}, expected length: {input_size}")

    if len(input_data) != input_size:
       print(f"Warning: input_data length is {len(input_data)}, expected {input_size}")
       return None , None
    
    input_tensor=torch.FloatTensor(input_data).unsqueeze(0).unsqueeze(0)

    with torch.no_grad():
       output=gru_model(input_tensor)

    probabilities=torch.softmax(output , dim=1).cpu().numpy()[0]  
    predicted_class=torch.argmax(output).item()
    
    return predicted_class , probabilities

# 비디오 파일 경로 지정 및 열기 
video_path="D:\\human_fall\\re_video\\training\\Y\\01452_O_B_FY_C8.mp4"
cap=cv2.VideoCapture(video_path)

# 비디오 속성 가져오기 
width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps=cap.get(cv2.CAP_PROP_FPS)

# 출력 비디오 설정 
fourcc=cv2.VideoWriter_fourcc(*'mp4v')
out_path='data_video_test_outputY_GRU_yolo.mp4'
out=cv2.VideoWriter(out_path,fourcc,fps,(width,height))

prev_landmarks=None

# 프레임 처리 루프 
while cap.isOpened():
    ret , frame=cap.read()
    
    if not ret:
       break

    rgb_frame=cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)
    results_pose=pose.process(rgb_frame)

    if results_pose.pose_landmarks:
        
        landmarks=np.array([[lm.x * width , lm.y * height , lm.z] for lm in results_pose.pose_landmarks.landmark])
       
        if prev_landmarks is not None: 
            result=detect_fall(landmarks , prev_landmarks)
            if result is not None:  
                label , probs=result 
                print(f"Predicted Class: {label}, Probabilities: {probs}")  
            else:
                print("Detection failed.")
        else: 
            label=None 

        prev_landmarks=landmarks 

        # 바운딩 박스와 라벨 그리기 
        if label is not None and bbox_info is not None and len(bbox_info) > 0:
            x1 , y1 , x2 , y2=bbox_info[0]   
            color=(0 ,255 ,0) if label==0 else ((255 ,255 ,0) if label==2 else (255 ,0 ,0)) 
            cv2.rectangle(frame , (int(x1), int(y1)), (int(x2), int(y2)), color ,2)
            class_name=class_names[label] if label is not None else 'Unknown'
            cv2.putText(frame , f'GRU: {class_name}' , (int(x1) , int(y1) -10) , cv2.FONT_HERSHEY_SIMPLEX ,0.7 , color ,2)
            print("YOLO results:", results[0].boxes.xyxy.cpu().numpy())
            print("Classes:", results[0].boxes.cls.cpu().numpy())
            print("Confidences:", results[0].boxes.conf.cpu().numpy())
        # 랜드마크 표시 
        mp_drawing.draw_landmarks(frame , results_pose.pose_landmarks , mp_pose.POSE_CONNECTIONS)

    # 프레임 저장 및 출력 
    resized_frame=cv2.resize(frame,(1920, 1080))
    out.write(resized_frame) 
    cv2.imshow('Fall Detection', resized_frame) 
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


0: 640x640 1 Fall, 6.0ms
Speed: 3.0ms preprocess, 6.0ms inference, 71.8ms postprocess per image at shape (1, 3, 640, 640)
Transformed coordinates: (2124.3585205078125, 799.4786682128906, 2524.6387939453125, 1467.958969116211)
Input data length: 27, expected length: 27
Predicted Class: 2, Probabilities: [   0.027933     0.17902     0.79305]


NameError: name 'bbox_info' is not defined

### GRU 모델만 사용
* bbox의 비율을 기준으로 클래스 분류

In [123]:
# 바운딩 박스 계산 및 그리기 함수
def calculate_and_draw_bbox(frame, landmarks):
    x_coordinates = landmarks[:, 0]
    y_coordinates = landmarks[:, 1]
    
    x1 = max(0, int(np.min(x_coordinates)))
    y1 = max(0, int(np.min(y_coordinates)))
    x2 = min(frame.shape[1], int(np.max(x_coordinates)))
    y2 = min(frame.shape[0], int(np.max(y_coordinates)))
    
    bbox_width = x2 - x1
    bbox_height = y2 - y1
    
    # 높이가 0일 경우 비율을 무한대로 설정
    bbox_ratio = bbox_width / bbox_height if bbox_height != 0 else float('inf')
    
    # 바운딩 박스 클래스 결정
    bbox_class = 0
    if bbox_ratio < 0.5:
        bbox_class = 0  # Normal
    elif 0.5 <= bbox_ratio < 0.7:
        bbox_class = 2  # Danger
    else:
        bbox_class = 1  # Fall
    
    # 바운딩 박스를 조금 더 넓게 조정 (각 방향으로 50픽셀 추가)
    padding = 50
    x1 = max(0, x1 - padding)
    y1 = max(0, y1 - padding)
    x2 = min(frame.shape[1], x2 + padding)
    y2 = min(frame.shape[0], y2 + padding)
    
    return (x1, y1, x2, y2), bbox_width, bbox_height, bbox_ratio, bbox_class

def calculate_head_upper_body_speed(keypoints, prev_keypoints):
    h = np.array([keypoints[0, 0], keypoints[0, 1]])  # 머리 좌표
    l = np.array([keypoints[11, 0], keypoints[11, 1]])  # 왼쪽 어깨 좌표
    r = np.array([keypoints[12, 0], keypoints[12, 1]])  # 오른쪽 어깨 좌표

    # 이전 프레임의 좌표
    prev_h = np.array([prev_keypoints[0, 0], prev_keypoints[0, 1]])
    prev_l = np.array([prev_keypoints[11, 0], prev_keypoints[11, 1]])
    prev_r = np.array([prev_keypoints[12, 0], prev_keypoints[12, 1]])

    # 현재 프레임과 이전 프레임의 상체 중심
    center_new = (h + l + r) / 3
    center_prev = (prev_h + prev_l + prev_r) / 3

    # 유클리드 거리 계산 (속도)
    speed = distance.euclidean(center_new, center_prev)
    return speed

# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

def process_landmarks(landmarks): 
    selected_landmarks = landmarks[LANDMARKS]
    return selected_landmarks[:, :2].flatten()

# GRU 모델 정의
class GRUModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, output_size=3, dropout=0.5):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

# GRU 모델 로드
input_size = 27  
hidden_size = 64
num_layers = 2
output_size = 3
dropout = 0.5    

gru_model = GRUModel(input_size, hidden_size, num_layers, output_size, dropout)
gru_model.load_state_dict(torch.load('D:\\project\\prjvenv\\GRU\\best_GRU_model.pt', map_location=torch.device('cpu')))
gru_model.eval()

# 클래스 이름 정의
class_names = {0: 'Normal', 1: 'Fall', 2: 'Danger'}

# 낙상 감지 함수
def detect_fall(landmarks, prev_landmarks, frame):
    speed = calculate_head_upper_body_speed(landmarks, prev_landmarks) if prev_landmarks is not None else 0
    processed_landmarks = process_landmarks(landmarks)
    
    bbox_info = calculate_and_draw_bbox(frame, landmarks)
    
    if bbox_info is None:
        return None
    
    bbox_width, bbox_height, bbox_ratio, bbox_class = bbox_info[1:4] + (bbox_info[4],)
    
    print(f"Processed landmarks length: {len(processed_landmarks)}")
    print(f"BBox width: {bbox_width}, height: {bbox_height}, ratio: {bbox_ratio}, speed: {speed}")   

    # processed_landmarks와 함께 바운딩 박스 좌표 및 속도 정보 추가
    input_data = np.concatenate([processed_landmarks,
                                  [bbox_width,
                                   bbox_height,
                                   bbox_ratio,
                                   speed,
                                   bbox_class]])
    
    if len(input_data) != input_size:
        print(f"Warning: input_data length is {len(input_data)}, expected {input_size}")
        return None
    
    input_tensor = torch.FloatTensor(input_data).unsqueeze(0).unsqueeze(0)
    
    with torch.no_grad():
        output = gru_model(input_tensor)

    probabilities = torch.softmax(output, dim=1).numpy()[0]
    predicted_class = torch.argmax(output).item()
    
    return predicted_class, probabilities

# 비디오 파일 경로 지정
video_path = "D:\\human_fall\\re_video\\validation\\N\\02327_H_A_N_C6.mp4"
# 비디오 파일 열기
cap = cv2.VideoCapture(video_path)

# 비디오 속성 가져오기
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 출력 비디오 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_path='data_video_test_outputY_GRU_only.mp4'
out= cv2.VideoWriter(out_path,fourcc,fps,(width,height))

prev_landmarks=None

# 프레임 처리 루프
while cap.isOpened():
    ret , frame= cap.read()
    if not ret:
        break

    # MediaPipe로 포즈 추정 
    rgb_frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_pose=pose.process(rgb_frame)

    if results_pose.pose_landmarks:
       landmarks=np.array([[lm.x * width , lm.y * height , lm.z] for lm in results_pose.pose_landmarks.landmark])
       
       # 바운딩 박스 계산 및 그리기 
       bbox_info=calculate_and_draw_bbox(frame , landmarks)

       if prev_landmarks is not None: 
           label , probs=detect_fall(landmarks , prev_landmarks , frame)
           print(f"Predicted Class: {label}, Probabilities: {probs}")  
       else: 
           label=None 

       prev_landmarks=landmarks 

       # 바운딩 박스와 라벨 그리기 
       x1 , y1 , x2 , y2=bbox_info[0]   
       color=(0 ,255 ,0) if label==0 else ((0, 255, 255) if label==1 else (255, 0, 0)) 
       cv2.rectangle(frame , (x1 , y1) , (x2 , y2) , color ,2)
       # 클래스 이름을 사용하여 텍스트 표시
       class_name = class_names[label] if label is not None else 'Unknown'
       cv2.putText(frame , f'GRU: {label}' , (x1 , y1 -10) , cv2.FONT_HERSHEY_SIMPLEX ,0.7 , color ,2)

       # 랜드마크 표시 
       mp_drawing.draw_landmarks(frame , results_pose.pose_landmarks , mp_pose.POSE_CONNECTIONS)

    # 프레임 저장 및 출력 
    resized_frame=cv2.resize(frame,(1920 ,1080))  
    out.write(resized_frame) 
    cv2.imshow('Fall Detection' , resized_frame) 
    if cv2.waitKey(1) & 0xFF==ord('q'):
       break

cap.release()
out.release()
cv2.destroyAllWindows()

Processed landmarks length: 22
BBox width: 439, height: 358, ratio: 1.2262569832402235, speed: 22.330371340889418
Predicted Class: 1, Probabilities: [   0.038052     0.73953     0.22242]
Processed landmarks length: 22
BBox width: 441, height: 394, ratio: 1.119289340101523, speed: 7.060977690259665
Predicted Class: 1, Probabilities: [   0.037483     0.74019     0.22233]
Processed landmarks length: 22
BBox width: 484, height: 417, ratio: 1.160671462829736, speed: 56.794296189125006
Predicted Class: 1, Probabilities: [   0.023751     0.77615      0.2001]
Processed landmarks length: 22
BBox width: 389, height: 371, ratio: 1.0485175202156334, speed: 38.520641438712275
Predicted Class: 1, Probabilities: [   0.038052     0.73953     0.22242]
Processed landmarks length: 22
BBox width: 486, height: 395, ratio: 1.230379746835443, speed: 18.204313168576896
Predicted Class: 1, Probabilities: [   0.023676     0.75962     0.21671]
Processed landmarks length: 22
BBox width: 567, height: 413, ratio: 1

### GRU 모델만 사용
* 속도를 우선적으로 클래스 분류

In [21]:
# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 랜드마크 인덱스 정의 (예: 코, 왼쪽 어깨, 오른쪽 어깨 등)
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]  # 총 11개 랜드마크

# GRU 모델 정의
class GRUModel(torch.nn.Module):
    def __init__(self, input_size=27):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size = 64
        self.num_layers = num_layers = 2
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True,
                          dropout=0.5)
        self.fc = nn.Linear(hidden_size, 3)  # output_size를 직접 지정합니다.
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

# GRU 모델 로드
input_size = 27
gru_model = GRUModel(input_size=input_size)  
gru_model.load_state_dict(torch.load('D:\\project\\prjvenv\\GRU\\best_GRU_model_2.pt', map_location=torch.device('cpu')))
gru_model.eval()

# 클래스 이름 정의
class_names = {0: 'Normal', 1: 'Fall', 2: 'Danger'}

# Threshold 값 정의
threshold_normal = 6.5   # 일반 상태로 간주되는 속도 임계값
threshold_danger = 10.5   # 위험 상태로 간주되는 속도 임계값

def calculate_head_upper_body_speed(keypoints, prev_keypoints):
    h = np.array([keypoints[0, 0], keypoints[0, 1]])   # 머리 좌표
    l = np.array([keypoints[11, 0], keypoints[11, 1]])  # 왼쪽 어깨 좌표
    r = np.array([keypoints[12, 0], keypoints[12, 1]])  # 오른쪽 어깨 좌표

    # 이전 프레임의 좌표가 없는 경우 속도는 0으로 설정
    if prev_keypoints is None:
        return 0.0

    prev_h = np.array([prev_keypoints[0, 0], prev_keypoints[0, 1]])
    prev_l = np.array([prev_keypoints[11, 0], prev_keypoints[11, 1]])
    prev_r = np.array([prev_keypoints[12, 0], prev_keypoints[12, 1]])

    # 현재 프레임과 이전 프레임의 상체 중심 계산
    center_new = (h + l + r) / 3
    center_prev = (prev_h + prev_l + prev_r) / 3

    # 유클리드 거리 계산 (속도)
    speed = distance.euclidean(center_new, center_prev)
    return speed

def process_landmarks(landmarks): 
    selected_landmarks = landmarks[LANDMARKS]   # 지정된 랜드마크 선택 
    return selected_landmarks[:, :2].flatten()   # (x,y) 좌표 반환

def calculate_and_draw_bbox(frame, landmarks):
    x_coordinates = landmarks[:, 0]
    y_coordinates = landmarks[:, 1]
    
    x1 = max(0, int(np.min(x_coordinates)))
    y1 = max(0, int(np.min(y_coordinates)))
    x2 = min(frame.shape[1], int(np.max(x_coordinates)))
    y2 = min(frame.shape[0], int(np.max(y_coordinates)))
    
    bbox_width = x2 - x1
    bbox_height = y2 - y1
    
    # 높이가 0일 경우 비율을 무한대로 설정
    bbox_ratio = bbox_width / bbox_height if bbox_height != 0 else float('inf')
    
    # 바운딩 박스를 조금 더 넓게 조정 (각 방향으로 패딩 추가)
    padding = 50
    x1 = max(0, x1 - padding)
    y1 = max(0, y1 - padding)
    x2 = min(frame.shape[1], x2 + padding)
    y2 = min(frame.shape[0], y2 + padding)

    return (x1, y1), (x2, y2), bbox_width, bbox_height

# 낙상 감지 함수
def detect_fall(landmarks, prev_landmarks):
    speed = calculate_head_upper_body_speed(landmarks, prev_landmarks)
    
    processed_landmarks = process_landmarks(landmarks)

    # 바운딩 박스 계산 및 그리기 
    top_left_bbox , bottom_right_bbox , bbox_width , bbox_height= calculate_and_draw_bbox(frame , landmarks)

    
    # 속도 기반 클래스 결정
    if speed < threshold_normal:
        bbox_class = 0   # Normal 
    elif speed < threshold_danger:
        bbox_class = 2   # Danger 
    else:
        bbox_class = 1   # Fall 

    return bbox_class

# 비디오 파일 경로 지정 및 열기 
video_path="D:\\human_fall\\re_video\\validation\\Y\\00170_H_A_SY_C5.mp4"
cap=cv2.VideoCapture(video_path)

# 비디오 속성 가져오기 
width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps=cap.get(cv2.CAP_PROP_FPS)

# 출력 비디오 설정 
fourcc=cv2.VideoWriter_fourcc(*'mp4v')
out_path='video_test_GRU_onlyspeed_8.mp4'
out=cv2.VideoWriter(out_path,fourcc,fps,(width,height))

prev_landmarks=None

# 프레임 처리 루프 
while cap.isOpened():
    ret , frame=cap.read()
    
    if not ret:
        break

    rgb_frame=cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)
    results_pose=pose.process(rgb_frame)

    if results_pose.pose_landmarks:
        landmarks=np.array([[lm.x * width , lm.y * height , lm.z] for lm in results_pose.pose_landmarks.landmark])
       
        if prev_landmarks is not None: 
            label=detect_fall(landmarks , prev_landmarks)  
            print(f"Predicted Class: {label}")  
        else: 
            label=None 

        prev_landmarks=landmarks 

        # 바운딩 박스와 라벨 그리기 
        top_left_bbox , bottom_right_bbox , _, _= calculate_and_draw_bbox(frame , landmarks)  
        color=(0 ,255 ,0) if label==0 else ((0 ,255, 255) if label==2 else (0, 0, 255)) 
        cv2.rectangle(frame , top_left_bbox , bottom_right_bbox , color ,2)
        class_name=class_names[label] if label is not None else 'Unknown'
        cv2.putText(frame , f'GRU: {class_name}' , (top_left_bbox[0] , top_left_bbox[1] -10) , cv2.FONT_HERSHEY_SIMPLEX ,0.7 , color ,2)

        # 랜드마크 표시 
        mp_drawing.draw_landmarks(frame , results_pose.pose_landmarks , mp_pose.POSE_CONNECTIONS)

    # 프레임 저장 및 출력 
    resized_frame=cv2.resize(frame,(1920 ,1080))  
    out.write(frame) 
    cv2.imshow('Fall Detection' , resized_frame) 
    if cv2.waitKey(1) & 0xFF==ord('q'):
         break

cap.release()
out.release()
cv2.destroyAllWindows()

Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Cl

### GRU 모델만 사용
* 속도 기반으로 초기 클래스 결정 후 bbox의 비율로 클래스 조정

In [46]:
# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 랜드마크 인덱스 정의 (예: 코, 왼쪽 어깨, 오른쪽 어깨 등)
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]  # 총 11개 랜드마크

# GRU 모델 정의
class GRUModel(torch.nn.Module):
    def __init__(self, input_size=27):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size = 64
        self.num_layers = num_layers = 2
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True,
                          dropout=0.5)
        self.fc = nn.Linear(hidden_size, 3)  # output_size를 직접 지정합니다.
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

# GRU 모델 로드
input_size = 27
gru_model = GRUModel(input_size=input_size)  
gru_model.load_state_dict(torch.load('D:\\project\\prjvenv\\GRU\\best_GRU_model_2.pt', map_location=torch.device('cpu')))
gru_model.eval()

# 클래스 이름 정의
class_names = {0: 'Normal', 1: 'Fall', 2: 'Danger'}

# Threshold 값 정의
threshold_normal = 6.5   # 일반 상태로 간주되는 속도 임계값
threshold_danger = 10.5   # 위험 상태로 간주되는 속도 임계값

def calculate_head_upper_body_speed(keypoints, prev_keypoints):
    h = np.array([keypoints[0, 0], keypoints[0, 1]])   # 머리 좌표
    l = np.array([keypoints[11, 0], keypoints[11, 1]])  # 왼쪽 어깨 좌표
    r = np.array([keypoints[12, 0], keypoints[12, 1]])  # 오른쪽 어깨 좌표

    # 이전 프레임의 좌표가 없는 경우 속도는 0으로 설정
    if prev_keypoints is None:
        return 0.0

    prev_h = np.array([prev_keypoints[0, 0], prev_keypoints[0, 1]])
    prev_l = np.array([prev_keypoints[11, 0], prev_keypoints[11, 1]])
    prev_r = np.array([prev_keypoints[12, 0], prev_keypoints[12, 1]])

    # 현재 프레임과 이전 프레임의 상체 중심 계산
    center_new = (h + l + r) / 3
    center_prev = (prev_h + prev_l + prev_r) / 3

    # 유클리드 거리 계산 (속도)
    speed = distance.euclidean(center_new, center_prev)
    return speed

def process_landmarks(landmarks): 
    selected_landmarks = landmarks[LANDMARKS]   # 지정된 랜드마크 선택 
    return selected_landmarks[:, :2].flatten()   # (x,y) 좌표 반환

def calculate_and_draw_bbox(frame, landmarks):
    x_coordinates = landmarks[:, 0]
    y_coordinates = landmarks[:, 1]
    
    x1 = max(0, int(np.min(x_coordinates)))
    y1 = max(0, int(np.min(y_coordinates)))
    x2 = min(frame.shape[1], int(np.max(x_coordinates)))
    y2 = min(frame.shape[0], int(np.max(y_coordinates)))
    
    bbox_width = x2 - x1
    bbox_height = y2 - y1
    
    # 높이가 0일 경우 비율을 무한대로 설정
    bbox_ratio = bbox_width / bbox_height if bbox_height != 0 else float('inf')
    
    # 바운딩 박스를 조금 더 넓게 조정 (각 방향으로 패딩 추가)
    padding = 50
    x1 = max(0, x1 - padding)
    y1 = max(0, y1 - padding)
    x2 = min(frame.shape[1], x2 + padding)
    y2 = min(frame.shape[0], y2 + padding)

    return (x1, y1), (x2, y2), bbox_width, bbox_height

# 낙상 감지 함수
def detect_fall(landmarks, prev_landmarks, fall_frame_counter): # 일정 프레임 이상 fall 클래스가 지속되면 fall 클래스 고정
    global determine_fall
    
    if determine_fall : 
        return 1, fall_frame_counter
    
    speed = calculate_head_upper_body_speed(landmarks, prev_landmarks)
    
    processed_landmarks = process_landmarks(landmarks)

    # 바운딩 박스 계산 및 그리기 
    top_left_bbox , bottom_right_bbox , bbox_width , bbox_height= calculate_and_draw_bbox(frame , landmarks)
    
    # 바운딩 박스 비율 계산
    bbox_ratio = bbox_width / bbox_height if bbox_height != 0 else float('inf')

    
    # 속도 기반 클래스 결정
    if speed < threshold_normal:
        bbox_class = 0   # Normal 
    elif speed < threshold_danger:
        bbox_class = 2   # Danger 
    else:
        bbox_class = 1   # Fall 
    
    # 바운딩 박스 비율에 따른 클래스 조정
    if bbox_class == 0 and bbox_ratio < 0.5 :
        bbox_class = 0   # Normal에서 Normal로 조정
    elif bbox_class == 0 and 0.5 <= bbox_ratio <= 0.7 : 
        bbox_class = 2   # Normal에서 Danger로 조정
    elif bbox_class == 0 and bbox_ratio > 1 : 
        bbox_class = 1   # Normal에서 Fall로 조정
        
    elif bbox_class == 2 and bbox_ratio < 0.5 : 
        bbox_class = 0
    elif bbox_class == 2 and 0.5 <= bbox_ratio <= 0.7 : 
        bbox_class = 2
    elif bbox_class == 2 and bbox_ratio > 1 : 
        bbox_class = 1
        
    elif bbox_class == 1 and bbox_ratio < 0.5 : 
        bbox_class = 0
    elif bbox_class == 1 and 0.5 <= bbox_ratio <= 0.7 : 
        bbox_class = 2
    elif bbox_class == 1 and bbox_ratio > 1 : 
        bbox_class = 1
        
    # Fall_counter 업데이트
    if bbox_class == 1 : 
        fall_frame_counter += 1
        if fall_frame_counter >= 10 : 
            determine_fall = True
    
    else : 
        fall_frame_counter = 0

    return bbox_class, fall_frame_counter

# 비디오 파일 경로 지정 및 열기 
video_path = "D:\\human_fall\\re_video\\validation\\Y\\02968_L_F_FY_C4.mp4"
cap=cv2.VideoCapture(video_path)

# 비디오 속성 가져오기 
width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps=cap.get(cv2.CAP_PROP_FPS)

# 출력 비디오 설정 
fourcc=cv2.VideoWriter_fourcc(*'mp4v')
out_path='GRU_speed+bboxratio_Y_8.mp4'
out=cv2.VideoWriter(out_path,fourcc,fps,(width,height))

prev_landmarks=None

# 프레임 처리 루프 
fall_frame_counter = 0
fall_threshold = 10
determine_fall = False

while cap.isOpened():
    ret , frame=cap.read()
    if not ret:
        break

    rgb_frame=cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)
    results_pose=pose.process(rgb_frame)

    if results_pose.pose_landmarks:
        landmarks=np.array([[lm.x * width , lm.y * height , lm.z] for lm in results_pose.pose_landmarks.landmark])
       
        if prev_landmarks is not None: 
            label, fall_frame_counter =detect_fall(landmarks , prev_landmarks, fall_frame_counter)  
            print(f"Predicted Class: {label}")  
        else: 
            label=None 

        prev_landmarks=landmarks 

        # 바운딩 박스와 라벨 그리기 
        top_left_bbox , bottom_right_bbox , _, _= calculate_and_draw_bbox(frame , landmarks)  
        color=(0 ,255 ,0) if label==0 else ((0 ,255, 255) if label==2 else (0, 0, 255)) 
        cv2.rectangle(frame , top_left_bbox , bottom_right_bbox , color ,2)
        class_name=class_names[label] if label is not None else 'Unknown'
        cv2.putText(frame , f'GRU: {class_name}' , (top_left_bbox[0] , top_left_bbox[1] -10) , cv2.FONT_HERSHEY_SIMPLEX ,0.7 , color ,2)

        if determine_fall : 
            cv2.putText(frame, 'FALL', (10, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255), 3)
        # 랜드마크 표시 
        mp_drawing.draw_landmarks(frame , results_pose.pose_landmarks , mp_pose.POSE_CONNECTIONS)

    # 프레임 저장 및 출력 
    resized_frame=cv2.resize(frame,(1920 ,1080))  
    out.write(frame) 
    cv2.imshow('Fall Detection' , resized_frame) 
    if cv2.waitKey(1) & 0xFF==ord('q'):
         break

cap.release()
out.release()
cv2.destroyAllWindows()

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Cl

In [16]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
import mediapipe as mp

# YOLO 모델 로드
yolo_model = YOLO('D:\\project\\prjvenv\\runs\\detect\\human_fall_s30\\weights\\best.pt')

# 낙상 감지 함수
def detect_fall(landmarks, bbox_width, bbox_height, bbox_ratio, confidence):
    processed_landmarks = process_landmarks(landmarks)
    
    # 랜드마크의 수를 확인
    if processed_landmarks.shape[0] != 22:  # 11개 랜드마크 * 2 (x, y)
        print(f"Processed landmarks count: {processed_landmarks.shape[0]}, expected 22")
    
    # 입력 데이터 생성 (confidence 포함)
    input_data = np.concatenate([processed_landmarks, [bbox_width, bbox_height, bbox_ratio, confidence]])
    
    # 입력 데이터 크기 확인
    if len(input_data) != 26:
        print(f"input_data length: {len(input_data)}, expected 26")
        return None, None
    
    input_tensor = torch.FloatTensor(input_data).unsqueeze(0).unsqueeze(0)

    with torch.no_grad():
        output = gru_model(input_tensor)
    
    probabilities = torch.softmax(output, dim=1).numpy()[0]
    predicted_class = torch.argmax(output, dim=1).item()
    
    print(f"Probabilities: Normal={probabilities[0]:.4f}, Danger={probabilities[1]:.4f}, Fall={probabilities[2]:.4f}")
    return predicted_class, probabilities

# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

def process_landmarks(landmarks): 
    selected_landmarks = landmarks[LANDMARKS]
    return selected_landmarks[:, :2].flatten()

# GRU 모델 로드
class GRUModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, output_size=3, dropout=0.5):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

input_size = 27 # 랜드마크 x,y 좌표 + 바운딩박스 비율 + confidence
hidden_size = 64
num_layers = 2
output_size = 3
dropout = 0.5    

gru_model = GRUModel(input_size, hidden_size, num_layers, output_size, dropout)
gru_model.load_state_dict(torch.load('D:\\project\\prjvenv\\GRU\\best_GRU_model_2.pt', map_location=torch.device('cpu')))
gru_model.eval()

# 비디오 파일 경로 지정
video_path = "D:\\human_fall\\re_video\\training\\Y\\02735_H_A_FY_C7.mp4"

# 비디오 파일 열기
cap = cv2.VideoCapture(video_path)

# 비디오 속성 가져오기
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

window_w = int(width * 0.3)
window_h = int(height * 0.3)

cv2.namedWindow('Fall Detection', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Fall Detection', window_w, window_h)
fps = cap.get(cv2.CAP_PROP_FPS)

# 출력 비디오 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('data_video_test_outputY.mp4', fourcc, fps, (width, height))

confidence_threshold = 0.3

previous_bbox = None
previous_label = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO 모델 실행
    results = yolo_model(frame)
    
    # YOLO 결과 처리
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy().astype(int)
        confidences = result.boxes.conf.cpu().numpy()
        class_ids = result.boxes.cls.cpu().numpy()
        
        if len(boxes) > 0:
            # YOLO가 객체를 감지한 경우 (바운딩 박스만 사용)
            box_indices_to_process = [i for i in range(len(boxes)) if confidences[i] > confidence_threshold]
            for i in box_indices_to_process:
                box = boxes[i]  # 각 객체의 바운딩 박스 처리
                confidence = confidences[i]
                class_id = class_ids[i]
                
                x1, y1, x2, y2 = box
                bbox_width = x2 - x1
                bbox_height = y2 - y1
                bbox_ratio = bbox_width / bbox_height
                
                # MediaPipe 처리
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results_pose = pose.process(rgb_frame)
                
                if results_pose.pose_landmarks:
                    landmarks = np.array([[lm.x * frame.shape[1], lm.y * frame.shape[0]] for lm in results_pose.pose_landmarks.landmark])
                    
                    try:
                        # GRU 모델을 통해 클래스 예측 (confidence를 포함하여 호출)
                        label, probs = detect_fall(landmarks, bbox_width, bbox_height, bbox_ratio ,confidence) 
                        previous_bbox = (x1,y1,x2,y2)
                        previous_label = label

                        # 바운딩 박스 색상 결정 (확률 기반으로 결정)
                        if probs[2] > 0.5:  # Fall 클래스 확률이 50% 이상일 때 빨간색으로 표시 (Fall은 빨간색으로 설정됨)
                            color = (0 ,0 ,255)   # Fall: Red
                        elif probs[1] > 0.5:   # Danger 클래스 확률이 50% 이상일 때 노란색으로 표시
                            color = (0 ,255 ,255)   # Danger: Yellow
                        else:
                            color = (0 ,255 ,0)   # Normal: Green

                        # 바운딩 박스 그리기 및 레이블 표시하기
                        cv2.rectangle(frame,(x1,y1),(x2,y2),color ,2) 
                        cv2.putText(frame,f'Class: {label}', (x1,y1 -10), cv2.FONT_HERSHEY_SIMPLEX ,0.7,color ,2)

                    except Exception as e:
                        print(f"GRU 모델 실행 중 에러: {e}")
                        label, probs = previous_label if previous_label is not None else (None,None)

                else:
                    # MediaPipe가 랜드마크를 감지하지 못한 경우 YOLO 결과 표시 (바운딩 박스만 그리기)
                    color_map_default = {0: (255 ,0 ,0), 1: (255 ,255 ,0)}  
                    color_default= color_map_default[class_id] if class_id in color_map_default else (255 ,255 ,255) 
                    cv2.rectangle(frame,(x1,y1),(x2,y2),color_default ,2)  
                    cv2.putText(frame,f"YOLO Class ID: {class_id}",(x1,y1 -30),cv2.FONT_HERSHEY_SIMPLEX ,0.7,color_default ,2)

    # 프레임 저장 및 출력
    out.write(frame)
    cv2.imshow('Fall Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 1 Fall, 74.9ms
Speed: 2.0ms preprocess, 74.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Fall, 3.0ms
Speed: 2.0ms preprocess, 3.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Fall, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Fall, 4.0ms
Speed: 2.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Fall, 3.0ms
Speed: 1.0ms preprocess, 3.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Fall, 3.0ms
Speed: 1.0ms preprocess, 3.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Fall, 3.0ms
Speed: 1.0ms preprocess, 3.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Fall, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Fall, 4.

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


0: 384x640 1 Fall, 21.0ms
Speed: 3.0ms preprocess, 21.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
GRU 모델 실행 중 에러: input.size(-1) must be equal to input_size. Expected 27, got 26

0: 384x640 1 Fall, 18.0ms
Speed: 3.0ms preprocess, 18.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)
GRU 모델 실행 중 에러: input.size(-1) must be equal to input_size. Expected 27, got 26

0: 384x640 1 Fall, 18.0ms
Speed: 3.0ms preprocess, 18.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)
GRU 모델 실행 중 에러: input.size(-1) must be equal to input_size. Expected 27, got 26

0: 384x640 1 Fall, 19.0ms
Speed: 5.0ms preprocess, 19.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
GRU 모델 실행 중 에러: input.size(-1) must be equal to input_size. Expected 27, got 26

0: 384x640 1 Fall, 24.0ms
Speed: 2.0ms preprocess, 24.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
GRU 모델 실행 중 에러: input.size(-1) must be equal to input_size. E